In [4]:
import pandas as pd
import numpy as np


df_path =  '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/info_finally.csv'
df = pd.read_csv(df_path)

df['returns'] = df['valor'].pct_change()

lags = [1,2,3,4,5,10]

for l in lags:
    df[f"lag_{l}"] = df['returns'].shift(l)


""" 
Creamos una linea de tendecias en base 5/10/20 dias si la media esperada (rolling_mean ) es positiva y alta
significa que la tendecia del el ultimo mes/semana/dias es que el dolar suba.

Luego si la desviacion estandar es muy volatil quiere decir que el dolar baja y sube constantemente... es decir: 
Si la desviacion estandar (rolling_std) es alta el dolar un dia puede subir un 5% y al otro un 20% y de repente al otro baja un 19%
Si la desviacion estandar es baja el mercado esta estable
"""
windows = [5,10,20]
for w in windows:
    df[f'rolling_mean_{w}'] = df['returns'].rolling(window=w).mean()
    df[f'rolling_std_{w}'] = df['returns'].rolling(window=w).std()

df.dropna(inplace=True)

df.to_csv('/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/feature_basic.csv')

print(df.columns)

Index(['fecha', 'valor', 'returns', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'lag_10', 'rolling_mean_5', 'rolling_std_5', 'rolling_mean_10',
       'rolling_std_10', 'rolling_mean_20', 'rolling_std_20'],
      dtype='object')


In [19]:
import pandas as pd
import numpy as np

def get_rsi(series_data, periodo=14):
    
    delta = series_data.diff()
    delta = delta.dropna()

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    
    avg_gain = gain.ewm(com=periodo-1, adjust=False).mean()
    avg_loss = loss.ewm(com=periodo-1, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


df_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/info_finally.csv'

df = pd.read_csv(df_path)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df.dropna(subset=['fecha'], inplace=True)


df.set_index('fecha', inplace=True)

df['rsi'] = get_rsi(df['valor'], periodo=14)
df['day_of_week'] = df.index.dayofweek
df['is_end_of_month'] = (df.index.day > 25).astype(int)
df.dropna(inplace=True)

save_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/features_advanced.csv'
df.to_csv(save_path) 



ValueError: cannot reindex on an axis with duplicate labels